In [7]:
# Import Dependcies
import pandas as pd
import hvplot.pandas

# Import API key
from api_keys import geoapify_key

In [8]:
# Load  CSV files to merge and then use to build map of US with shooting points
df1 = pd.read_csv("Outputs/clean_1_updated.csv")
df2 = pd.read_csv("Outputs/clean_2_updated.csv")
df3 = pd.read_csv("Outputs/clean_3_updated.csv")

# Display the data to make sure the relative paths are working
df1.head()
#df2
#df3.head()

,CDCODE,year,school,city,state,address,urbanrural,race,killed,injured,victims,date,type,NCESSCH,time,Latitude,Longitude
0,1000.0,2009,William Penn High School,New Castle,Delaware,713 E Basin Rd,suburbtown,BLACK,0,1,1,8-Jan-09,High School,1.000230e+11,8:30PM,39.662057,-75.566313
1,1701.0,2009,Dunbar Vocational Career Academy,Chicago,Illinois,3000 S King Dr,urban,BLACK,0,5,5,9-Jan-09,High School,1.709930e+11,8:00PM,41.875562,-87.624421
2,1712.0,2009,Cahokia High School,Cahokia,Illinois,800 Range Ln,suburbtown,BLACK,1,0,1,23-Jan-09,High School,1.708040e+11,7:20PM,38.566161,-90.179424
3,3713.0,2009,Zebulon Middle School,Zebulon,North Carolina,Halifax Road and New Oak Street,suburbtown,BLACK,0,1,1,11-Feb-09,Middle School,3.704720e+11,6:00AM,35.822162,-78.313213
4,NaN,2009,Central High School,Detroit,Michigan,2425 Tuxedo St,urban,NaN,0,1,1,17-Feb-09,High School,2.601103e+11,2:20PM,42.331551,-83.046640


In [27]:
# Test merging
merged_df = pd.merge(df2, df3, on=["Latitude", "Longitude", "School"], how="right")
merged_df
final_df = pd.merge(merged_df, df1, on=["Latitude", "Longitude"], how="right")
final_df

,School,Year,Date_x,Time_x,City_x,State_x,Killed,Injured,Latitude,Longitude,...,address,urbanrural,race,killed,injured,victims,date,type,NCESSCH,time
0,Dover High School,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.662057,-75.566313,...,713 E Basin Rd,suburbtown,BLACK,0,1,1,8-Jan-09,High School,1.000230e+11,8:30PM
1,William Penn High School,2023.0,1/10/2023,11:40 AM,New Castle,Delaware,0.0,0.0,39.662057,-75.566313,...,713 E Basin Rd,suburbtown,BLACK,0,1,1,8-Jan-09,High School,1.000230e+11,8:30PM
2,William Penn High School,2023.0,1/10/2023,11:40 AM,New Castle,Delaware,0.0,0.0,39.662057,-75.566313,...,713 E Basin Rd,suburbtown,BLACK,0,1,1,8-Jan-09,High School,1.000230e+11,8:30PM
3,Harlan High School,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.875562,-87.624421,...,3000 S King Dr,urban,BLACK,0,5,5,9-Jan-09,High School,1.709930e+11,8:00PM
4,Camlumet High School,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.875562,-87.624421,...,3000 S King Dr,urban,BLACK,0,5,5,9-Jan-09,High School,1.709930e+11,8:00PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353,Woodrow Wilson High School,2010.0,4/28/2010,12:15 PM,Portsmouth,Virginia,0.0,0.0,36.832008,-76.297699,...,4841 Clifford St,urban,WHITE,0,1,1,20-Nov-18,Elementary School,5.103000e+11,4:00PM
1354,Douglass Park Elementary School,2018.0,3/19/2018,4:23 PM,Portsmouth,Virginia,0.0,1.0,36.832008,-76.297699,...,4841 Clifford St,urban,WHITE,0,1,1,20-Nov-18,Elementary School,5.103000e+11,4:00PM
1355,Simonsdale Elementary School,2018.0,11/20/2018,3:45 PM,Portsmouth,Virginia,0.0,1.0,36.832008,-76.297699,...,4841 Clifford St,urban,WHITE,0,1,1,20-Nov-18,Elementary School,5.103000e+11,4:00PM
1356,Manor High School,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.832008,-76.297699,...,4841 Clifford St,urban,WHITE,0,1,1,20-Nov-18,Elementary School,5.103000e+11,4:00PM


In [21]:
#final_df["Date_x"].value_counts()

Date_x
1/10/2023     4
2/4/2008      4
12/11/2009    4
12/6/2005     4
2/17/2009     4
             ..
2/23/2010     1
4/28/2010     1
5/18/2010     1
8/30/2010     1
5/24/2022     1
Name: count, Length: 260, dtype: int64

In [30]:
%%capture --no-display

# Creating test map prior to merging DataFrames
map_plot_1 = final_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = True,
    frame_width = 800,
    frame_height = 600,
    size = "victims",
    color = "City_x",
    hover_cols = ["State", "School"],
    alpha = 0.9,
    scale = 5
)

map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City_x,victims,School)

In [18]:
# Save the plot as a PNG
hvplot.save(map_plot_1, 'Outputs/school_shooting_plot.png')